# ***Random Orbits starting at periapsis - one period, variable dt, dp = 800***

In [2]:
import sys
import pickle
import torch
import numpy as np
from random import randint
from numpy import linalg as LA
import matplotlib.pyplot as plt
import matplotlib as mpl  

# from scipy.integrate

# function to output the state space dynamics of the orbit
def orbitdyn2D(t,x,mu):
    r = LA.norm([x[0], x[1]]) # magnitude of position vector
    ax = -mu*x[0]/r**3 # acceleration in x-dir based on two-body problem
    ay = -mu*x[1]/r**3 # acceleration in y-dir based on two-body problem

    fdyn = np.array([x[2], x[3], ax, ay])
    return fdyn


# runge-kutta fourth-order numerical integration
def rk4(func, tk, _yk, uk, _dt, **kwargs):
    """
    single-step fourth-order numerical integration (RK4) method
    func: system of first order ODEs
    tk: current time step
    _yk: current state vector [y1, y2, y3, ...]
    _dt: discrete time step size
    **kwargs: additional parameters for ODE system
    returns: y evaluated at time k+1
    """

    # evaluate derivative at several stages within time interval
    f1 = func(tk, _yk, uk, **kwargs)
    f2 = func(tk + _dt / 2, _yk + (f1 * (_dt / 2)), uk, **kwargs)
    f3 = func(tk + _dt / 2, _yk + (f2 * (_dt / 2)), uk, **kwargs)
    f4 = func(tk + _dt, _yk + (f3 * _dt), uk, **kwargs)

    # return an average of the derivative over tk, tk + dt
    return np.array(_yk + (_dt / 6) * (f1 + (2 * f2) + (2 * f3) + f4))


device = torch.device("cuda:0")

number_of_initial_conditions = 30
dp = 800
nx = 4 # state space representation of the model has 4 states
m = 2 # number of control states
predict_time = 15

# creating empty vectors to store data
data_x = torch.zeros((0,dp-predict_time+1, nx))
data_xdot = torch.zeros((0,dp-predict_time+1, nx))
data_y = torch.zeros((0,dp-predict_time+1, nx))
data_y2 = torch.zeros((0,dp-predict_time+1, nx))

# looping through all the initial conditions
for num in range(number_of_initial_conditions):

    # Two Body Problem parameters
    G = 6.6742e-20 # Gravitational constant [km^3*kg^-1*s^-2]
    r_Earth = 6378.14  # Average radius of Earth [km]
    m1 = 5.974e24 # mass of Earth [kg]
    m2 = 1000 # mass of satellite [kg]
    mu = G*(m1 + m2) # gravitational parameter [km^3*s^-2]

    # ~~~~~~~~~~~~~~~ Orbital Dynamics ~~~~~~~~~~~~~~~# 
    e = 0.0
    rp = r_Earth + randint(200, 5000) # radius of Perigee [km]

    ra = (rp + e*rp)/(1-e)
    a = (rp + ra)/2 # Semi-Major axis [km]
    period = 2*np.pi*(a**3/mu)**0.5 # Period of Orbit

    # initial condition (start at periapsis)
    x = rp
    y = 0
    x_dot = 0
    y_dot = (mu*((2/rp) - (1/a)))**0.5
    
    x0_2D = torch.tensor([x, y, x_dot, y_dot])
    time = np.linspace(0, int(period), dp) # simulate for dp=1000 data points
    dt = int(period)/dp
    xk = x0_2D
    X = np.empty([nx,])
    XDOT = np.empty([nx,])
    Y = np.empty([nx,])

    for t in time:
        X = np.vstack((X, xk))
        XDOT = np.vstack((XDOT,orbitdyn2D(t,xk,mu)))
        xk = rk4(orbitdyn2D, t, xk, mu, dt) # RK4 approximation of solution
        Y = np.vstack((Y, xk))

    X = torch.FloatTensor(X)
    XDOT = torch.FloatTensor(XDOT)
    Y = torch.FloatTensor(Y)

    newx = X[1:-(predict_time-1),:]
    newxdot = XDOT[1:-(predict_time-1),:]
    newy = Y[1:-(predict_time-1),:]
    newy2 = Y[predict_time:,:]

    # storing in one large data array
    data_x = torch.vstack([data_x, newx[None,:]])  
    data_xdot = torch.vstack([data_xdot, newxdot[None,:]]) 
    data_y = torch.vstack([data_y, newy[None,:]])
    data_y2 = torch.vstack([data_y2, newy2[None,:]])

min_i = torch.argmin(data_x[:,0,0])
max_i = torch.argmax(data_x[:,0,0])

training_lim = torch.stack((data_x[min_i,:,0:2], data_x[max_i,:,0:2]),dim=2)

dp = dp - (predict_time-1)
with open('twobody_variables_data_3D.pkl', 'wb') as f:
    pickle.dump([dp,dt,nx,m,predict_time,e], f)
torch.save(data_x, "X_Data.pt")
torch.save(data_xdot, "Xdot_Data.pt")
torch.save(data_y, "Y_Data.pt")
torch.save(data_y2, "Y2_Data.pt")
torch.save(training_lim, "Training_Plot.pt")